# Minicurso Prático: Vector Database com Qdrant

Neste notebook, vamos aprender na prática como interagir com um banco de dados vetorial, o **Qdrant**. Cobriremos o ciclo completo de operações, desde a criação de uma "coleção" até a execução de buscas semânticas inteligentes.

**O que vamos fazer:**
1.  **Conectar** ao nosso banco de dados Qdrant rodando no Docker.
2.  **Preparar um modelo de IA** para transformar texto em vetores (embeddings).
3.  **Criar uma coleção** para armazenar nossos vetores.
4.  **Inserir dados** (documentos de texto sobre IA, culinária, ciência e história).
5.  **Realizar buscas** que entendem o significado, não apenas palavras-chave.
6.  **Filtrar resultados** combinando busca semântica e metadados.
7.  **Gerenciar os dados** com operações de deleção e contagem.

## Passo 1: Importando as Bibliotecas e Conectando ao Banco

Vamos importar as bibliotecas necessárias para o nosso trabalho e estabelecer a conexão com a instância do Qdrant que está rodando no Docker.

- `qdrant_client`: Lib para "conversar" com o Qdrant.
- `sentence_transformers`: Lib que contém o modelo de IA para gerar os vetores.

In [3]:
# Importações
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
import numpy as np #importante para manipulação de vetores

# --- Conexão com o Cliente Qdrant ---
# Se o Qdrant está rodando via `docker-compose up`, ele estará acessível neste endereço.
try:
    client = QdrantClient(host="localhost", port=6333)
    print("Conexão com o Qdrant estabelecida com sucesso!")
    print("Versão do Qdrant:", client.get_collections()) # teste para verificar se a conexão está funcionando
except Exception as e:
    print(f"Falha ao conectar com o Qdrant. Verifique se o contêiner Docker está rodando.")
    print(f"Erro: {e}")

Conexão com o Qdrant estabelecida com sucesso!
Versão do Qdrant: collections=[CollectionDescription(name='documentos_ia')]


## Passo 2: Preparando o Modelo de IA e a Coleção

Agora que estamos conectados, precisamos de duas coisas antes de inserir dados:

1.  **Um modelo de IA:** Ele será nossa "ferramenta" para transformar textos (que o banco não entende) em vetores numéricos (que o banco entende). Usaremos um modelo pré-treinado da biblioteca `Sentence Transformers`.
2.  **Uma "coleção" no Qdrant:** É o espaço, similar a uma tabela, onde nossos vetores e metadados serão armazenados.

Vamos carregar o modelo e definir as configurações da nossa coleção.

In [4]:
# Carrega o modelo pré-treinado 'all-MiniLM-L6-v2' da biblioteca SentenceTransformer.
# Este modelo é leve e eficiente, ótimo para demonstrações. Ele é o responsável
# por converter nossas frases em vetores.
model = SentenceTransformer('all-MiniLM-L6-v2')

# Precisamos saber o tamanho (a dimensionalidade) dos vetores que o modelo gera.
# Para este modelo, o tamanho é 384.
vector_size = model.get_sentence_embedding_dimension()

# Por fim, definimos o nome da nossa coleção.
collection_name = "documentos_ia"

print(f"Modelo '{model.__class__.__name__}' carregado.")
print(f"Tamanho do vetor (dimensionalidade): {vector_size}")
print(f"Nome da coleção que será criada: '{collection_name}'")

Modelo 'SentenceTransformer' carregado.
Tamanho do vetor (dimensionalidade): 384
Nome da coleção que será criada: 'documentos_ia'


Com as configurações prontas, vamos efetivamente criar a **coleção** no Qdrant.

O código abaixo primeiro tenta deletar a coleção se ela já existir. Isso serve para que possamos rodar o notebook várias vezes desde o início sem receber um erro de "coleção já existe".

In [6]:
#`try/except` para deletar a coleção se ela já existir
try:
    client.delete_collection(collection_name=collection_name)
    print(f"Coleção '{collection_name}' antiga encontrada e deletada para garantir um início limpo.")
except Exception as e:
    print(f"Coleção '{collection_name}' não existia previamente. Tudo certo para continuar.")

# Cria a nova coleção com a configuração de vetores que definimos
client.create_collection(
    collection_name=collection_name, #documentos_ia
    vectors_config=models.VectorParams(
        size=vector_size,      # o tamanho do vetor (384)
        distance=models.Distance.COSINE  # a métrica de similaridade de cosseno
    ),
)

print(f"Coleção '{collection_name}' criada com sucesso no Qdrant!")

Coleção 'documentos_ia' antiga encontrada e deletada para garantir um início limpo.
Coleção 'documentos_ia' criada com sucesso no Qdrant!


## Passo 3: Inserindo Dados na Coleção

Com a nossa coleção vazia criada, vamos agora para etapa de 'escrita' no banco de dados. Vamos pegar uma lista de frases simples, com temas variados, e prepará-las para serem inseridas.

Cada 'item' que inserimos no Qdrant é chamado de **ponto** (Point). Um `ponto` é a estrutura fundamental e é composto por 3 partes principais:
- `id`: Um identificador único para o ponto (como uma chave primária).
- `vector`: O vetor numérico gerado pelo nosso modelo de IA. É a representação matemática do dado.
- `payload`: Um dicionário com dados extras que queremos associar ao vetor (metadados), como o texto original, categorias, datas, etc.

In [7]:
# Primeiro, definimos a nossa lista de documentos em um formato de dicionário Python.
# Cada dicionário contém o texto a ser vetorizado e uma categoria como metadado.
documents = [
    # Tecnologia
    {"text": "A inteligência artificial está revolucionando a medicina com diagnósticos mais precisos.", "category": "tecnologia"},
    {"text": "Grandes modelos de linguagem (LLMs) são a base de muitos chatbots e assistentes virtuais.", "category": "tecnologia"},
    {"text": "O aprendizado de máquina é um subcampo da IA focado em algoritmos que aprendem com dados.", "category": "tecnologia"},
    {"text": "A computação em nuvem permite o armazenamento e processamento de dados em larga escala.", "category": "tecnologia"},
    {"text": "Cibersegurança é a prática de proteger sistemas e redes contra ataques digitais.", "category": "tecnologia"},

    # Culinária
    {"text": "A culinária italiana é famosa por suas massas frescas e pizzas de forno a lenha.", "category": "culinaria"},
    {"text": "O sushi é um prato tradicional japonês feito de arroz temperado, peixe cru e algas.", "category": "culinaria"},
    {"text": "A moqueca capixaba é um cozido de peixe e frutos do mar típico do Espírito Santo, Brasil.", "category": "culinaria"},
    {"text": "Os croissants franceses são conhecidos por sua massa folhada crocante e amanteigada.", "category": "culinaria"},
    {"text": "Cozinhar sous-vide é uma técnica que utiliza um banho de água a temperatura controlada.", "category": "culinaria"},

    # História
    {"text": "O Império Romano foi uma das civilizações mais influentes da história ocidental.", "category": "historia"},
    {"text": "A Segunda Guerra Mundial foi um conflito global que durou de 1939 a 1945.", "category": "historia"},
    {"text": "Cleópatra foi a última faraó do Antigo Egito, conhecida por sua inteligência e alianças políticas.", "category": "historia"},
    {"text": "A Proclamação da República no Brasil ocorreu em 15 de novembro de 1889.", "category": "historia"},
    {"text": "As pirâmides de Gizé serviram como tumbas para os faraós Quéops, Quéfren e Miquerinos.", "category": "historia"},

    # Ciência
    {"text": "A teoria da relatividade de Albert Einstein mudou nossa compreensão sobre espaço e tempo.", "category": "ciencia"},
    {"text": "O telescópio espacial James Webb consegue observar galáxias formadas logo após o Big Bang.", "category": "ciencia"},
    {"text": "A molécula de DNA contém as instruções genéticas para o desenvolvimento dos seres vivos.", "category": "ciencia"},
    {"text": "A fotossíntese é o processo pelo qual as plantas convertem luz solar em energia química.", "category": "ciencia"},
    {"text": "Buracos negros são regiões do espaço com um campo gravitacional extremamente forte.", "category": "ciencia"},
]

print(f"Temos {len(documents)} documentos de exemplo para inserir.")

Temos 20 documentos de exemplo para inserir.


Agora, vamos iterar (fazer um loop) sobre cada documento da lista acima. Para cada um, faremos duas coisas:

1.  **Gerar o Embedding:** Usar nosso modelo (`model`) para transformar o texto em um vetor de 384 dimensões.
2.  **Criar o Ponto:** Montar a estrutura do `PointStruct` do Qdrant com o ID, o vetor gerado e o payload (nossos metadados).

In [8]:
# Inicializamos uma lista vazia para guardar nossos pontos
points = []

# Loop para percorrer os documentos e transformá-los em pontos
for idx, doc in enumerate(documents):
    # Gerar o embedding: pega o texto e o transforma em um vetor numérico.
    embedding = model.encode(doc["text"]).tolist()
    
    # Cria a estrutura do ponto com ID, vetor e metadados (payload).
    points.append(
        models.PointStruct(
            id=idx,  # Usamos o índice do loop como um ID único
            vector=embedding,
            payload={
                "original_text": doc["text"],
                "category": doc["category"]
            }
        )
    )

# Ao final do loop, a lista 'points' conterá todos os nossos dados prontos para serem enviados.
print(f"Estrutura de dados para {len(points)} pontos foi criada com sucesso.")
print("\nExemplo do primeiro ponto a ser inserido:")
print(points[0])

c:\Users\55819\Desktop\projetos\banco-de-dados\seminario\VectorDataBase\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Estrutura de dados para 20 pontos foi criada com sucesso.

Exemplo do primeiro ponto a ser inserido:
id=0 vector=[-0.0058999331668019295, -0.009198222309350967, -0.04568072780966759, -0.0062173232436180115, -0.08498112857341766, -0.03501074016094208, 0.014892986975610256, 0.11612645536661148, 0.034215763211250305, 0.023821623995900154, 0.05144776031374931, 0.015415575355291367, -0.023634886369109154, -0.000549100514035672, -0.1196724995970726, -0.026842832565307617, 0.03396619111299515, 0.012623627670109272, 0.03486590087413788, 0.034491702914237976, 0.02379419468343258, -0.006574203725904226, -0.06870835274457932, 0.0774228572845459, -0.0926598459482193, 0.04005620628595352, 0.025128668174147606, -0.08153228461742401, -0.011863509193062782, -0.042686495929956436, 0.08866962045431137, 0.07166998833417892, 0.09888964146375656, -0.060623899102211, -0.0051544662564992905, -0.0407172366976738, 0.07826758176088333, -0.029718605801463127, 0.025781283155083656, 0.0735730230808258, -0.05651209

Com nossa lista de pontos pronta, usamos um único comando, `upsert`, para enviar todos eles para o Qdrant de uma vez. O comando `upsert` é bom porque ele insere novos pontos ou atualiza os que já existem (caso o ID já esteja no banco).

In [9]:
# Envia a lista de pontos para a nossa coleção no Qdrant.
operation_info = client.upsert(
    collection_name=collection_name,
    wait=True,  # Pede para a operação esperar a conclusão antes de continuar
    points=points,
)

print("Dados inseridos no Qdrant!")
print("\nInformação da operação retornada pelo servidor:")
print(operation_info)

Dados inseridos no Qdrant!

Informação da operação retornada pelo servidor:
operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


## Passo 4: Realizando Buscas Semânticas

Com nossos dados no banco, é hora de consultá-los. Diferente de uma busca tradicional em SQL com `SELECT abc FROM xyz`, uma busca vetorial encontra os resultados mais "próximos" em significado.

O processo é simples:
1.  Definimos uma frase de busca (nossa "pergunta").
2.  Usamos o **mesmo modelo de IA** para converter essa frase em um vetor.
3.  Enviamos esse vetor para o Qdrant.
4.  O Qdrant calcula a "distância" (usando a métrica de cosseno que definimos) entre o nosso vetor de busca e todos os vetores da coleção e nos retorna os mais próximos, ou seja, os mais similares.

### 4.1. Exemplo 1: Busca Semântica Pura

Vamos começar com uma busca sobre **"modelos de IA para conversação"**. A palavra "chatbot", por exemplo, não está na nossa frase de busca, mas está em um dos documentos que inserimos.

In [10]:
# 1. Definimos a nossa busca
query_text_1 = "modelos de IA para conversação"

# 2. Convertemos a busca em um vetor
query_embedding_1 = model.encode(query_text_1).tolist()

# 3. Executamos a busca no Qdrant
search_result_1 = client.search(
    collection_name=collection_name,
    query_vector=query_embedding_1,
    limit=3,  # pedimos os 3 resultados mais próximos
    with_payload=True  # dizemos para incluir os metadados
)

# 4. Exibimos os resultados de forma legível
print(f"--- Resultados da busca por: '{query_text_1}' ---")
for hit in search_result_1:
    print(f"\nID do Ponto: {hit.id}")
    print(f"Similaridade (Score): {hit.score:.4f}")
    print(f"  Texto Original: {hit.payload['original_text']}")
    print(f"  Categoria: {hit.payload['category']}")

--- Resultados da busca por: 'modelos de IA para conversação' ---

ID do Ponto: 1
Similaridade (Score): 0.4952
  Texto Original: Grandes modelos de linguagem (LLMs) são a base de muitos chatbots e assistentes virtuais.
  Categoria: tecnologia

ID do Ponto: 3
Similaridade (Score): 0.4240
  Texto Original: A computação em nuvem permite o armazenamento e processamento de dados em larga escala.
  Categoria: tecnologia

ID do Ponto: 4
Similaridade (Score): 0.4093
  Texto Original: Cibersegurança é a prática de proteger sistemas e redes contra ataques digitais.
  Categoria: tecnologia


C:\Users\55819\AppData\Local\Temp\ipykernel_11228\363025758.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result_1 = client.search(


### 4.2. Exemplo 2: Combinando Busca Semântica com Filtros

Esta é uma das funcionalidades mais úteis dos bancos de dados vetoriais modernos. E se quisermos encontrar textos sobre um assunto, mas apenas dentro de uma **categoria específica**?

Vamos buscar por **"comida boa"**, mas restringindo a busca **apenas** para os documentos da categoria `culinaria`. Isso evita que o resultado traga, por exemplo, um texto sobre a "boa" estratégia de um imperador romano.

In [11]:
# 1. Definimos a nova busca
query_text_2 = "comida boa"

# 2. Convertemos em um vetor
query_embedding_2 = model.encode(query_text_2).tolist()

# 3. Executamos a busca, mas desta vez adicionando um filtro
search_result_2 = client.search(
    collection_name=collection_name,
    query_vector=query_embedding_2,
    # AQUI ESTÁ O FILTRO: definimos uma condição obrigatória (must)
    # onde o campo 'category' no payload deve ter o valor 'culinaria'.
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="category",
                match=models.MatchValue(value="culinaria")
            )
        ]
    ),
    limit=2, # Pedimos os 2 melhores resultados dentro do filtro
    with_payload=True
)

# 4. Exibimos os resultados
print(f"--- Resultados da busca por: '{query_text_2}' (filtrado por Categoria: culinaria) ---")
for hit in search_result_2:
    print(f"\nID do Ponto: {hit.id}")
    print(f"Similaridade (Score): {hit.score:.4f}")
    print(f"  Texto Original: {hit.payload['original_text']}")
    print(f"  Categoria: {hit.payload['category']}")

--- Resultados da busca por: 'comida boa' (filtrado por Categoria: culinaria) ---

ID do Ponto: 7
Similaridade (Score): 0.3236
  Texto Original: A moqueca capixaba é um cozido de peixe e frutos do mar típico do Espírito Santo, Brasil.
  Categoria: culinaria

ID do Ponto: 5
Similaridade (Score): 0.2892
  Texto Original: A culinária italiana é famosa por suas massas frescas e pizzas de forno a lenha.
  Categoria: culinaria


C:\Users\55819\AppData\Local\Temp\ipykernel_11228\1328935485.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result_2 = client.search(


## Passo 5: Gerenciando os Dados (Retrieve, Delete e Count)

Além de buscar por similaridade, que é a função principal, podemos também realizar operações diretas nos dados. Por exemplo:
- Buscar um item que já conhecemos pelo seu ID.
- Remover dados que não são mais necessários ou estão desatualizados.
- Verificar quantos itens temos no total.

### 5.1. Buscando um Ponto Específico por ID (`retrieve`)

Imagine que você já sabe o ID de um documento e quer apenas os dados dele, sem fazer uma busca por similaridade. Para isso, usamos o comando `retrieve`, que é extremamente rápido.

In [12]:
# Vamos buscar o ponto que tem o ID 2
point_id_to_retrieve = 2

retrieved_points = client.retrieve(
    collection_name=collection_name,
    ids=[point_id_to_retrieve],
    with_payload=True # incluir os metadados
)

print("Resultado:")
print(retrieved_points)

Resultado:
[Record(id=2, payload={'original_text': 'O aprendizado de máquina é um subcampo da IA focado em algoritmos que aprendem com dados.', 'category': 'tecnologia'}, vector=None, shard_key=None, order_value=None)]


### 5.2. Deletando Pontos da Coleção (`delete`)

Agora, vamos simular a remoção de alguns dados. A operação `delete` permite remover um ou mais pontos de forma eficiente, bastando fornecer seus IDs. Vamos deletar os pontos com ID 3 e 4 (os de culinária).

In [13]:
# Deleta os pontos com IDs 3 e 4 da nossa coleção.
operation_info_delete = client.delete(
    collection_name=collection_name,
    points_selector=models.PointIdsList(points=[3, 4])
)

print("--- Deletando os pontos com ID 3 e 4 ---")
print("Operação de deleção concluída!")
print(operation_info_delete)

--- Deletando os pontos com ID 3 e 4 ---
Operação de deleção concluída!
operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


### 5.3. Verificando o Estado Final da Coleção (`count`)

Depois de deletar podemos confirmar que a operação funcionou usando o comando `count` para ver quantos pontos restaram na coleção. Começamos com 20, deletamos 2, então o resultado deve ser 18.

In [14]:
# Conta o número de pontos restantes na coleção
count_result = client.count(
    collection_name=collection_name,
    exact=True # Pede uma contagem exata
)

print("--- Contando o total de pontos restantes ---")
print(f"Total de pontos na coleção '{collection_name}': {count_result.count}")

--- Contando o total de pontos restantes ---
Total de pontos na coleção 'documentos_ia': 18


## Conclusão

Completamos o ciclo de vida de interação com um banco de dados vetorial. Agora sabemos como:
- **Configurar e conectar** a um banco de dados vetorial.
- **Criar uma coleção** para armazenar vetores.
- **Inserir dados** usando embeddings.
- **Realizar buscas** semânticas e com filtros.
- **Gerenciar os dados** com operações de busca por ID, deleção e contagem.

### Caso de Uso Final

Criamos uma aplicação web interativa simples com Streamlit (`src/app.py`). Vamos focar em usar a operação de `search` para criar uma experiência de busca.

Teste no terminal: **streamlit run src/app.py**